# Transfer images to dataframe for trainning

In [1]:
import pandas as pd 
import numpy as np
from scipy.misc import imread, imshow, imresize
import os

# Read images and save to a dataframe

In [2]:
img_path = 'data/images/'

In [3]:
def read_img(path):
    img = imread(path)
    return imresize(img, [224,224,3])

In [4]:
folder = os.listdir(img_path)
folder.sort()
img_list = [read_img(img_path+img) for img in folder]

/home/leo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/home/leo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()


In [5]:
img_data = pd.DataFrame({'date':folder, 'img':img_list})

# Add time for merging with readings from the station

In [6]:
img_data['Year_img'] = img_data['date'].apply(lambda x: int(x[7:11]))
img_data['Month_img'] = img_data['date'].apply(lambda x: int(x[11:13]))
img_data['Day_img'] = img_data['date'].apply(lambda x: int(x[13:15]))
img_data['Time_img'] = img_data['date'].apply(lambda x: int(x[15:17]))
img_data = img_data.drop('date', axis = 1)
img_data = img_data[['Year_img', 'Month_img', 'Day_img', 'Time_img', 'img']]

# Save images first

In [7]:
img_data.to_pickle('data/imgs.pkl')

# Read in readings and images dataframes

In [8]:
img_data = pd.read_pickle('data/imgs.pkl')
weather = pd.read_csv('data/weather_data.csv')

# Merging

In [9]:
data = pd.merge(weather, img_data, how='left', 
                left_on=['Year', 'Month', 'Day', 'Time'], 
                right_on=['Year_img', 'Month_img', 'Day_img', 'Time_img'])

In [10]:
data = data.dropna(axis=0, how='any')
data = data.reset_index()
data = data.drop(['index', 'Year_img', 'Month_img', 'Day_img', 'Time_img'], axis=1)

In [11]:
data.to_pickle('data/final_data.pkl')